In [1]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
from ping3 import ping
import random
import json
import re
import multiprocess as mp

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'/Users/dongzheng/Codes/PythonWorkspace/CSE5005-Project/data'

In [3]:
with open("./website.json", "r", encoding="utf8") as f:
    website_list = json.load(f)

website_list[:10]

[{'id': '2205',
  'name': '人人网',
  'logo': 'http://reg007.u.qiniudn.com/app/renren.jpg',
  'desc': '人人网，中国领先的实名制SNS社交网络。加入人人网，找到老同学，结识新朋友。',
  'category': '社交',
  'home': 'http://www.renren.com/'},
 {'id': '2217',
  'name': '途牛旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/tuniu.jpg',
  'desc': '途牛旅游网-中国专业全面的旅游网,客户满意度94%;提供自助游(自由行),国内旅游,出境旅游,自驾游,公司旅游。低价保证,专业服务,九大出游保障,服务超百万人出游。',
  'category': '旅行',
  'home': 'http://www.tuniu.com/'},
 {'id': '2224',
  'name': '酷讯旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/kuxun.jpg',
  'desc': '酷讯旅游网(Kuxun.cn)是中国领先的在线旅游媒体。为您提供低价可靠的国内国际机票，特价机票99元起，另提供全国20万家酒店预订查询、酒店评论，最低2折起，以及列车时刻表、50000多旅游景点的旅游攻略信息，为您的度假提供最佳的旅游线路。',
  'category': '旅行',
  'home': 'http://www.kuxun.cn/'},
 {'id': '2246',
  'name': '金山词霸',
  'logo': 'http://reg007.u.qiniudn.com/app/iciba.jpg',
  'desc': '爱词霸英语为广大网友提供在线翻译、在线词典、英语口语、英语学习资料、汉语词典，金山词霸下载等服务，致力于为您提供优质权威的在线英语服务，是5000万英语学习者的首选。',
  'category': '工具',
  'home': 'http://www.iciba.com/'},
 {'id': '2247',
  'name': '一听音乐网',
  '

In [4]:
pattern = re.compile("http://(.+?)/")
url_list = [re.findall(pattern, dic["home"])[0] for dic in website_list]

len(url_list)
url_list[:10]

212

['www.renren.com',
 'www.tuniu.com',
 'www.kuxun.cn',
 'www.iciba.com',
 'www.1ting.com',
 'www.youku.com',
 'www.zol.com.cn',
 'www.baihe.com',
 'www.izhenxin.com',
 'www.taobao.com']

In [4]:
with open("./links.txt", "r", encoding="utf8") as f:
    links=f.read().splitlines()
    
links[:5]

['https://github.com',
 'https://docs.github.com',
 'https://skills.github.com',
 'https://resources.github.com',
 'https://support.github.com']

In [12]:
url_list = [link.split("//")[1].split("/")[0] for link in links]

len(url_list)
url_list[:10]

2921

['github.com',
 'docs.github.com',
 'skills.github.com',
 'resources.github.com',
 'support.github.com',
 'githubstatus.com',
 'twitter.com',
 'partner.github.com',
 'desktop.github.com',
 'services.github.com']

In [14]:
url = random.choice(url_list)
delay = ping(url)

url
delay

'www.tmrow.com'

0.018360614776611328

In [16]:
delete_url=[]
for url in url_list:
    try:
        if not ping(url):
            delete_url.append(url)
    except Exception:
        delete_url.append(url)
        
for url in delete_url:
    url_list.remove(url)

len(url_list)

2385

In [17]:
with open("./links_filtered.json", "w") as f:
    json.dump(url_list, f)

In [19]:
with open("./links_filtered.json", "r", encoding="utf8") as f:
    url_list = json.load(f)
    
len(url_list)
url_list[:5]

2385

['docs.github.com',
 'skills.github.com',
 'resources.github.com',
 'support.github.com',
 'githubstatus.com']

In [7]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [8]:
df_user = pd.DataFrame(columns=["user_id", "address", "url"])
for i in range(2000):
    url = random.choice(url_list)
    # while not ping(url):
    #     url=random.choice(url_list)
    df_user.loc[i] = [i, gen_addr(), url]

df_user

,user_id,address,url
0,0,1MPtDy6rUY8U8LPFDtQGxPUng6rwJq3Gfj,www.51talk.com
1,1,19RsbK1QtwvC5fiMcvCdsncR6VvqMLvYQs,www.meituan.com
2,2,16aHLEnVLGR32Dh1N8otWqVEmgxenSH8TZ,www.letv.com
3,3,1CzAJmLPnGAuTUquyp7gzY5g65EqchoWam,www.51.com
4,4,1NrJoQKuUKHhvKT4eN3zMrLC13mVJRjRz3,www.vko.cn
...,...,...,...
1995,1995,17rahWV1NFaMqKfJpGH1Di3PHQqTQ5z3tY,www.yeepay.com
1996,1996,1D8xGuhLojFGPV4Qm7vptWUxoU79ua8c1b,www.tujia.com
1997,1997,1NWnwq1wskNmWc1PFaUVXdMGjFUtguUzPF,www.huanqiu.com
1998,1998,17q2zh9zcqoAvtodjxTHAxww3KSFVcCBe6,www.chinahr.com


In [9]:
df_user.to_csv("user.csv", index=False)

---

In [12]:
!python gen_tx.py > gen_data.log 2>&1

In [13]:
df_tx = pd.read_pickle("./tx_99771.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-25 09:48:30.414545,2022-10-25 09:48:30.432322,0.017777,0,1KYxPXSLA4s6eQAAFm3849g3mksDE4CWFK,1HRUm7h7EyP4qXm4JrBd3M1ko8GyQHXKdo,5.112375
1,2022-10-25 09:48:30.331380,2022-10-25 09:48:30.434821,0.103441,1,1GywTZkeUMWf9J3NaJhCbq6LtSeUkC6Rid,17aUdKKWkBuoDtzGUcfXnmwgtL4imsxv9Z,8.107424
2,2022-10-25 09:48:30.383802,2022-10-25 09:48:30.435350,0.051548,0,12gtzJc3qrQThEk3TcUxCukmF6CfbEMdBm,1PpHffFCJa8PLMRUsksLi2KaCWATrS78zi,0.261991
3,2022-10-25 09:48:30.438316,2022-10-25 09:48:30.478155,0.039839,0,13LLxejwz7YKCnrDAvGA5a1V1r36vtJoXc,18MXbFhYMmcorHBdh5WJRCDUNwd5pTRiPg,0.848824
4,2022-10-25 09:48:30.358478,2022-10-25 09:48:30.526710,0.168232,1,1HCMQasEvtSK9Lm5S5MbJzYUJj7jCs1j2E,12qYZB54ici8jVgLAWap4dDtNRM77qQdj4,1.843508
...,...,...,...,...,...,...,...
99766,2022-10-25 11:09:51.654577,2022-10-25 11:09:51.694776,0.040199,0,1JtVnzq5hhKFzgNhUbzBe4N6qY5LW1pXhv,19Jb2yVGpvJxkxN98fcp4XJLn1uCMaksTj,8.893335
99767,2022-10-25 11:09:51.689212,2022-10-25 11:09:51.746803,0.057591,0,1G7jLPXC4DPYQ4BXTPacuAEAqJQBx3sJHJ,1KfBqnpyazENxBDR736jCMH9yroPnR6vr3,7.300961
99768,2022-10-25 11:09:51.721649,2022-10-25 11:09:51.761165,0.039516,0,1C7U3FErAtCJWeVRmYtZbd4uh2eXzJGjVG,1PhWATQnDnVSLAMa93k8mKpDaNzdQ1PhtJ,9.941156
99769,2022-10-25 11:09:51.747805,2022-10-25 11:09:51.763729,0.015924,0,1BSi3qJE4LnCXaUNof7GQ6PqmpyYjeWZCE,1F4BzsrrbJjYcU5fTYmSgJazSpVicutV75,1.019992


In [14]:
sum(df_tx["label"] == 1)

20777

---

---

Generate training data

In [15]:
df_tx = pd.read_pickle("./tx_99771.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-25 09:48:30.414545,2022-10-25 09:48:30.432322,0.017777,0,1KYxPXSLA4s6eQAAFm3849g3mksDE4CWFK,1HRUm7h7EyP4qXm4JrBd3M1ko8GyQHXKdo,5.112375
1,2022-10-25 09:48:30.331380,2022-10-25 09:48:30.434821,0.103441,1,1GywTZkeUMWf9J3NaJhCbq6LtSeUkC6Rid,17aUdKKWkBuoDtzGUcfXnmwgtL4imsxv9Z,8.107424
2,2022-10-25 09:48:30.383802,2022-10-25 09:48:30.435350,0.051548,0,12gtzJc3qrQThEk3TcUxCukmF6CfbEMdBm,1PpHffFCJa8PLMRUsksLi2KaCWATrS78zi,0.261991
3,2022-10-25 09:48:30.438316,2022-10-25 09:48:30.478155,0.039839,0,13LLxejwz7YKCnrDAvGA5a1V1r36vtJoXc,18MXbFhYMmcorHBdh5WJRCDUNwd5pTRiPg,0.848824
4,2022-10-25 09:48:30.358478,2022-10-25 09:48:30.526710,0.168232,1,1HCMQasEvtSK9Lm5S5MbJzYUJj7jCs1j2E,12qYZB54ici8jVgLAWap4dDtNRM77qQdj4,1.843508
...,...,...,...,...,...,...,...
99766,2022-10-25 11:09:51.654577,2022-10-25 11:09:51.694776,0.040199,0,1JtVnzq5hhKFzgNhUbzBe4N6qY5LW1pXhv,19Jb2yVGpvJxkxN98fcp4XJLn1uCMaksTj,8.893335
99767,2022-10-25 11:09:51.689212,2022-10-25 11:09:51.746803,0.057591,0,1G7jLPXC4DPYQ4BXTPacuAEAqJQBx3sJHJ,1KfBqnpyazENxBDR736jCMH9yroPnR6vr3,7.300961
99768,2022-10-25 11:09:51.721649,2022-10-25 11:09:51.761165,0.039516,0,1C7U3FErAtCJWeVRmYtZbd4uh2eXzJGjVG,1PhWATQnDnVSLAMa93k8mKpDaNzdQ1PhtJ,9.941156
99769,2022-10-25 11:09:51.747805,2022-10-25 11:09:51.763729,0.015924,0,1BSi3qJE4LnCXaUNof7GQ6PqmpyYjeWZCE,1F4BzsrrbJjYcU5fTYmSgJazSpVicutV75,1.019992


In [16]:
df_user=pd.read_csv("./user.csv")

user_list=df_user["address"].values.tolist()

user_list[:5]

['1MPtDy6rUY8U8LPFDtQGxPUng6rwJq3Gfj',
 '19RsbK1QtwvC5fiMcvCdsncR6VvqMLvYQs',
 '16aHLEnVLGR32Dh1N8otWqVEmgxenSH8TZ',
 '1CzAJmLPnGAuTUquyp7gzY5g65EqchoWam',
 '1NrJoQKuUKHhvKT4eN3zMrLC13mVJRjRz3']

In [17]:
user_list.index("17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m")

1

In [17]:
def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [18]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data

,send_timestamp,recv_timestamp,latency,label,from,to,value,from_user_id,to_user_id
0,2022-10-25 09:48:30.414545,2022-10-25 09:48:30.432322,0.017777,0,1KYxPXSLA4s6eQAAFm3849g3mksDE4CWFK,1HRUm7h7EyP4qXm4JrBd3M1ko8GyQHXKdo,5.112375,1907,227
1,2022-10-25 09:48:30.331380,2022-10-25 09:48:30.434821,0.103441,1,1GywTZkeUMWf9J3NaJhCbq6LtSeUkC6Rid,17aUdKKWkBuoDtzGUcfXnmwgtL4imsxv9Z,8.107424,60,1090
2,2022-10-25 09:48:30.383802,2022-10-25 09:48:30.435350,0.051548,0,12gtzJc3qrQThEk3TcUxCukmF6CfbEMdBm,1PpHffFCJa8PLMRUsksLi2KaCWATrS78zi,0.261991,1583,1241
3,2022-10-25 09:48:30.438316,2022-10-25 09:48:30.478155,0.039839,0,13LLxejwz7YKCnrDAvGA5a1V1r36vtJoXc,18MXbFhYMmcorHBdh5WJRCDUNwd5pTRiPg,0.848824,1514,1682
4,2022-10-25 09:48:30.358478,2022-10-25 09:48:30.526710,0.168232,1,1HCMQasEvtSK9Lm5S5MbJzYUJj7jCs1j2E,12qYZB54ici8jVgLAWap4dDtNRM77qQdj4,1.843508,1112,900
...,...,...,...,...,...,...,...,...,...
99766,2022-10-25 11:09:51.654577,2022-10-25 11:09:51.694776,0.040199,0,1JtVnzq5hhKFzgNhUbzBe4N6qY5LW1pXhv,19Jb2yVGpvJxkxN98fcp4XJLn1uCMaksTj,8.893335,1178,1168
99767,2022-10-25 11:09:51.689212,2022-10-25 11:09:51.746803,0.057591,0,1G7jLPXC4DPYQ4BXTPacuAEAqJQBx3sJHJ,1KfBqnpyazENxBDR736jCMH9yroPnR6vr3,7.300961,426,1148
99768,2022-10-25 11:09:51.721649,2022-10-25 11:09:51.761165,0.039516,0,1C7U3FErAtCJWeVRmYtZbd4uh2eXzJGjVG,1PhWATQnDnVSLAMa93k8mKpDaNzdQ1PhtJ,9.941156,1964,209
99769,2022-10-25 11:09:51.747805,2022-10-25 11:09:51.763729,0.015924,0,1BSi3qJE4LnCXaUNof7GQ6PqmpyYjeWZCE,1F4BzsrrbJjYcU5fTYmSgJazSpVicutV75,1.019992,329,1816


In [19]:
df_data.drop(columns=["send_timestamp", "from", "to"], inplace=True)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
0,2022-10-25 09:48:30.432322,0.017777,0,5.112375,1907,227
1,2022-10-25 09:48:30.434821,0.103441,1,8.107424,60,1090
2,2022-10-25 09:48:30.435350,0.051548,0,0.261991,1583,1241
3,2022-10-25 09:48:30.478155,0.039839,0,0.848824,1514,1682
4,2022-10-25 09:48:30.526710,0.168232,1,1.843508,1112,900
...,...,...,...,...,...,...
99766,2022-10-25 11:09:51.694776,0.040199,0,8.893335,1178,1168
99767,2022-10-25 11:09:51.746803,0.057591,0,7.300961,426,1148
99768,2022-10-25 11:09:51.761165,0.039516,0,9.941156,1964,209
99769,2022-10-25 11:09:51.763729,0.015924,0,1.019992,329,1816


In [20]:
# df_data["recv_timestamp"]=df_data["recv_timestamp"].apply(lambda x: x.value)

# df_data

In [21]:
df_data.to_pickle(f"./data_{len(df_data)}.pkl")

In [22]:
df_data[["recv_timestamp", "from_user_id", "to_user_id", "value", "label"]].values[:5]

array([[Timestamp('2022-10-25 09:48:30.432322'), 1907, 227,
        5.112375488707563, 0],
       [Timestamp('2022-10-25 09:48:30.434821'), 60, 1090,
        8.10742405244109, 1],
       [Timestamp('2022-10-25 09:48:30.435350'), 1583, 1241,
        0.26199133937441, 0],
       [Timestamp('2022-10-25 09:48:30.478155'), 1514, 1682,
        0.8488239081643234, 0],
       [Timestamp('2022-10-25 09:48:30.526710'), 1112, 900,
        1.8435082598156494, 1]], dtype=object)